In [1]:
import pandas as  pd
import snappy
import numpy as np


### <span style="color:#DC143C">**1. Diretórios Mapeados**</span>

In [2]:
source_file = '../../Data/Raw/dados_treinoteste.csv'
save_file = '../../Data/Processed/dados_treinoteste.parquet'


### <span style="color:#DC143C">**2.Transferência Dados para DataFrame**</span>

In [3]:
dataframe = pd.read_csv(source_file, sep='|',encoding = "utf-8")

#### <span style="color:#4B0082">**2.1 Formato do DataFrame**</span>

In [4]:
if (dataframe.empty) :
    print('DATAFRAME VAZIO')
else:
  print('shape:', dataframe.shape)
  print('columns:', dataframe.columns)
   

shape: (22718, 4)
columns: Index(['id', 'din_solicitacao', 'dsc_texto', 'tip_supreal'], dtype='object')


#### <span style="color:#4B0082">**2.2 Remover Colunas "Não Úteis" para Modelo**</span>

In [5]:
dataframe = dataframe.drop(columns=['id','din_solicitacao'])
#print('columns:', dataframe.columns)

#### <span style="color:#4B0082">**2.3 Quantidade de valores não nulos**</span>

In [6]:
dataframe.count()


dsc_texto      22718
tip_supreal    22717
dtype: int64

#### <span style="color:#4B0082">**2.4  Valores nulos**</span>

In [7]:
dataframe.isnull().sum()


dsc_texto      0
tip_supreal    1
dtype: int64

### <span style="color:#DC143C">**3.Tratamento de Dados no DataFrame**</span>

#### <span style="color:#4B0082">**3.1 Remover linhas com valores null em qualquer coluna**</span>

In [8]:
#dataframe=dataframe.dropna(subset=['TEXTO'])
dataframe = dataframe.dropna()
dataframe.isnull().sum()


dsc_texto      0
tip_supreal    0
dtype: int64

#### <span style="color:#4B0082">**3.2 Remover duplicatas** </span>

In [9]:
dataframe = dataframe.drop_duplicates()


### <span style="color:#DC143C">**4.Tratamento de Caracteres**</span>

#### <span style="color:#4B0082">**4.1 UPPER - Letras minusculas** </span>

In [10]:
dataframe['dsc_texto'] = dataframe['dsc_texto'].str.lower()

In [11]:
dataframe['tip_supreal'] = dataframe['tip_supreal'].str.lower()

#### <span style="color:#4B0082">**4.2 Substuir o caracteres "Indesejados" por espaços**</span>

In [19]:
def corrigir_nomes(nome):
    nome = nome.replace('0',' ').replace('1',' ').replace('2',' ').replace('3',' ').replace('4',' ').replace('5',' ').replace('6',' ').replace('7',' ').replace('8',' ').replace('9',' ')
    nome = nome.replace('º',' ').replace('ª',' ').replace('²',' ').replace('°',' ')
    nome = nome.replace('à','a').replace('ã','a').replace('á','a').replace('â','a').replace('é','e').replace('ê','e').replace('í','i').replace('ó','o').replace('õ','o').replace('ô','o')
    return nome

dataframe['dsc_texto'] = dataframe['dsc_texto'].apply(corrigir_nomes)

In [12]:
dataframe['dsc_texto'] = dataframe['dsc_texto'].str.replace(r'[^\w\s]+', ' ')
dataframe['dsc_texto'] = dataframe['dsc_texto'].str.replace(r'ú','u')
dataframe['dsc_texto'] = dataframe['dsc_texto'].str.replace(r'ü','u')
dataframe['dsc_texto'] = dataframe['dsc_texto'].str.replace(r'ç','c')




#### <span style="color:#4B0082">**4.3 "Visão" dos dados do Dataframe**</span>

In [13]:
dataframe.head(5)

,dsc_texto,tip_supreal
0,realizar testes nos controles de potencia ativ...,prop
1,equipe snefabbdorbp atualizacao de firmware u...,prop
2,inspecao ao longo da lt apos desarme,oper
3,manutencao na bomba de extracao de condensado ...,oper
4,intervencao para realizacao de atividades dos ...,oper


### <span style="color:#DC143C">**5.Resumo Estatístico da cadeia de caracteres**</span>

#### <span style="color:#4B0082">**5.1 Resumo sobre as colunas**</span>

In [14]:
dataframe.describe(include='all')

# Descreve caracteristicas de 1 coluna apenas
#dataframe['dsc_manutencao'].describe()

# Descreve apenas colunas "Numericas"
#dataframe.describe(include=[np.number])

# Descreve apenas colunas Textp
#dataframe.describe(exclude=[np.number])

,dsc_texto,tip_supreal
count,17738,17738
unique,16554,2
top,implantacao da bd do sage,oper
freq,43,8926


#### <span style="color:#4B0082">**5.2 Variável Alvo**</span>

In [15]:
dataframe.groupby('tip_supreal').count()

,dsc_texto
tip_supreal,
oper,8926
prop,8812


#### <span style="color:#4B0082">**5.3 Variáveis Preditoras**</span>

In [16]:
dataframe.dsc_texto.str.len().describe()

count    17738.000000
mean       164.578250
std        211.914361
min          1.000000
25%         75.000000
50%        113.000000
75%        179.000000
max       6261.000000
Name: dsc_texto, dtype: float64

### <span style="color:#DC143C">**5.Exportar Base de Dados PARQUET**</span>

In [17]:
dataframe.to_parquet(save_file, compression='snappy')